# Sentiment Analysis Notebook

In [2]:
#import natural language processing toolkit

import nltk as nltk

In [3]:
#download sample tweets

nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/corissa.mantooth/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

SyntaxError: invalid syntax (<ipython-input-7-648cbcc9e6cc>, line 1)

### -Tokenizing the data

In [8]:
#import samples

from nltk.corpus import twitter_samples

In [9]:
#label three datasets

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [12]:
#download punkt : a pretrained model to help tokenize words and sentences

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/corissa.mantooth/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
#use the .tokenized() method to tokenize positive tweets

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [14]:
#now check it out

print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


### -Normalizing the Data

In [ ]:
#Normalizing in NLP converts words to canonical form. ex: run, running, ran, runs == run (all various forms of run

#Stemming : heuristic process that removes ends of words: it removes affixes from a word
#Lemmatization : analyzes the structure of the wrod and context to convert to normalized form...takes more time, but more accurate

In [15]:
#download wordnet: lexical database that helps the script determine the base word
#download averaged_perceptron_tagger: determine context of a word

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/corissa.mantooth/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/corissa.mantooth/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [17]:
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples #didn't I do this?

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


 Meanings: NNP: Noun, proper, singular
 NN: Noun, common, singular or mass
 IN: Preposition or conjunction, subordinating
 VBG: Verb, gerund or present participle
 VBN: Verb, past participle

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


### -Removing Noise from Data

In [ ]:
#Remove all noise. In this case it is things like 'stop words', hyperlinks, twitter handles in replies, punctuation and special characters. Basically anything that doesn't add value


In [23]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)", "", token)
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
#This is creating a function that removes anything with the http at the begining that is part of the url and replace it with '' (an empty string) instead
#This also normalizes the data the same way as the previous cell

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/corissa.mantooth/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(remove_noise(tweet_tokens[0], stop_words))


['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [29]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [30]:
#compare cleaned and not

print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


### -Determining Word Density

In [ ]:
#what words occur most frequently?

In [32]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
        
all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [33]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))
#prints 10 most common words/terms and count

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


### -Preparing for the model

In [ ]:
#convert tokens to a dictionary then split to train-test

In [34]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)
        
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


In [35]:
import random
positive_dataset = [(tweet_dict, "Positive")
                   for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                   for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)
train_data = dataset[:7000]
test_data = dataset[7000:]
#The last two lines split the shuffled data into 70-30 for training and testing

### -Building and Testing Model

In [37]:
from nltk import classify
from nltk import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.9973333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2042.0 : 1.0
                      :) = True           Positi : Negati =   1665.4 : 1.0
                follower = True           Positi : Negati =     37.5 : 1.0
                    glad = True           Positi : Negati =     20.6 : 1.0
                     x15 = True           Negati : Positi =     18.8 : 1.0
                     sad = True           Negati : Positi =     17.7 : 1.0
               community = True           Positi : Negati =     17.2 : 1.0
                   enjoy = True           Positi : Negati =     14.1 : 1.0
                  arrive = True           Positi : Negati =     14.0 : 1.0
           unfortunately = True           Negati : Positi =     12.2 : 1.0
None


In [38]:
#play around to test it's accuracy

from nltk.tokenize import word_tokenize

custom_tweet = "It’s a free country. He’s perfectly free to starve himself to death."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive
